### Importando as bibliotecas

In [84]:
import django
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "MovieRecomendation.settings")
django.setup()

In [85]:
from recommendations.utils import get_all_movies, search_movies, search_movie_by_id, get_genre_mapping

In [86]:
movies = get_all_movies()['results']

In [87]:
print(movies)

[{'adult': False, 'backdrop_path': '/gJL5kp5FMopB2sN4WZYnNT5uO0u.jpg', 'genre_ids': [28, 12, 16, 35, 10751], 'id': 1011985, 'original_language': 'en', 'original_title': 'Kung Fu Panda 4', 'overview': 'Po is gearing up to become the spiritual leader of his Valley of Peace, but also needs someone to take his place as Dragon Warrior. As such, he will train a new kung fu practitioner for the spot and will encounter a villain called the Chameleon who conjures villains from the past.', 'popularity': 3781.812, 'poster_path': '/wkfG7DaExmcVsGLR4kLouMwxeT5.jpg', 'release_date': '2024-03-02', 'title': 'Kung Fu Panda 4', 'video': False, 'vote_average': 6.9, 'vote_count': 159}, {'adult': False, 'backdrop_path': '/deLWkOLZmBNkm8p16igfapQyqeq.jpg', 'genre_ids': [14, 12, 28], 'id': 763215, 'original_language': 'en', 'original_title': 'Damsel', 'overview': "A young woman's marriage to a charming prince turns into a fierce fight for survival when she's offered up as a sacrifice to a fire-breathing drag

In [88]:
query = 'Inception'
search_results = search_movies(query)

In [89]:
print(search_results)

{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg', 'genre_ids': [28, 878, 12], 'id': 27205, 'original_language': 'en', 'original_title': 'Inception', 'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.', 'popularity': 112.341, 'poster_path': '/edv5CZvWj09upOsy2Y6IwDhK8bt.jpg', 'release_date': '2010-07-15', 'title': 'Inception', 'video': False, 'vote_average': 8.4, 'vote_count': 35410}, {'adult': False, 'backdrop_path': '/JeGkRdNsOuMrgwBdtB0hp763MU.jpg', 'genre_ids': [18, 53], 'id': 613092, 'original_language': 'es', 'original_title': 'El crack cero', 'overview': 'Madrid, Spain, 1975; shortly after the end of the Franco dictatorship. Six months after the mysterious death of his lover, a prestigious tailor

In [90]:
movie_id = 27205
search_result_id = search_movie_by_id(movie_id)

In [91]:
print(search_result_id)

{'adult': False, 'backdrop_path': '/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg', 'belongs_to_collection': None, 'budget': 160000000, 'genres': [{'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 12, 'name': 'Adventure'}], 'homepage': 'https://www.warnerbros.com/movies/inception', 'id': 27205, 'imdb_id': 'tt1375666', 'original_language': 'en', 'original_title': 'Inception', 'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.', 'popularity': 112.341, 'poster_path': '/edv5CZvWj09upOsy2Y6IwDhK8bt.jpg', 'production_companies': [{'id': 923, 'logo_path': '/8M99Dkt23MjQMTTWukq4m5XsEuo.png', 'name': 'Legendary Pictures', 'origin_country': 'US'}, {'id': 9996, 'logo_path': '/3tvBqYsBhxWeHlu62SIJ1el93O7.png', 'name': 'Syncopy', 'origin_country

In [92]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [93]:
genres = [movie['genre_ids'] for movie in search_results['results']]
votes = [[movie['vote_average']] for movie in search_results['results']]

mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(genres)

scaler = StandardScaler()
votes_scaled = scaler.fit_transform(votes)

features = np.hstack((genres_encoded, votes_scaled))

In [94]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)
model_knn.fit(features)
distances, indices = model_knn.kneighbors([features[0]], n_neighbors=5)
similar_movies_titles = [search_results['results'][i]['title'] for i in indices.flatten()]
print(similar_movies_titles)

['Inception', 'Inception: The Cobol Job', 'Bikini Inception', 'The Crack: Inception', 'Inception: Music from the Motion Picture']


In [95]:
genre_mapping = get_genre_mapping()
print(genre_mapping)
unique_genre_names = sorted(set(genre_mapping[genre_id] for movie in movies for genre_id in movie['genre_ids'] if genre_id in genre_mapping))

{28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}


In [96]:
def genres_vector(movie):
    vector = np.zeros(len(unique_genre_names))
    for genre_id in movie['genre_ids']:
        if genre_id in genre_mapping:
            genre_name = genre_mapping[genre_id]
            genre_index = unique_genre_names.index(genre_name)
            vector[genre_index] = 1
    return vector

In [97]:
tfidf = TfidfVectorizer(stop_words='english')
overviews = tfidf.fit_transform([movie['overview'] for movie in movies])

In [98]:
def prepare_movie_data(movies, vectorizer=None):
    if vectorizer is None:
        vectorizer = TfidfVectorizer(stop_words='english')
        overviews = [movie.get('overview', '') for movie in movies]  
        tfidf_matrix = vectorizer.fit_transform(overviews)
    else:
        tfidf_matrix = vectorizer.transform([movies.get('overview', '')])
    return tfidf_matrix, vectorizer

In [99]:
movies = get_all_movies()['results']  

all_movies_data, vectorizer = prepare_movie_data(movies)

movie_id = 27205
target_movie = search_movie_by_id(movie_id)

In [100]:
target_movie_data, _ = prepare_movie_data(target_movie, vectorizer)

similarities = cosine_similarity(target_movie_data, all_movies_data)

most_similar_indices = np.argsort(similarities[0])[::-1][1:21]

In [101]:
for index in most_similar_indices:
    print(movies[index]['title'])

Argylle
Poor Things
Wonka
Dune: Part Two
Code 8 Part II
No Way Up
Sri Asih
The Pig, the Snake and the Pigeon
Megamind vs. the Doom Syndicate
Jaque Mate
Death Alley
Land of Bad
Vikings: Battle of Heirs
Migration
Anyone But You
Oppenheimer
Ruthless
Meteor
Kung Fu Panda 4
